In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
PoseLandmark = mp.solutions.pose.PoseLandmark
mp_pose = mp.solutions.pose
np.set_printoptions(precision=4, suppress=True)

In [ ]:
def compute_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
    return np.degrees(angle)


In [ ]:
def detect_heel_strike(prev_y, current_y, threshold=0.005):
    if prev_y is None:
        return False
    velocity = current_y - prev_y
    return -threshold < velocity < threshold

In [ ]:
def compute_stride_length(prev_pos, curr_pos):
    if prev_pos is None:
        return 0.0
    prev = np.array(prev_pos)
    curr = np.array(curr_pos)
    return float(np.linalg.norm(curr - prev))

In [ ]:
def extract_frame_landmarks(landmark_list):
    d = {}
    for idx, lm_enum in enumerate(PoseLandmark):
        name = lm_enum.name
        lm = landmark_list[idx]
        d[f"{name}_x"] = float(lm.x)
        d[f"{name}_y"] = float(lm.y)
        d[f"{name}_z"] = float(lm.z)
        d[f"{name}_visibility"] = float(lm.visibility)

        d['left_knee_angle'] = compute_angle([d['LEFT_HIP_x'], d['LEFT_HIP_y']], [d['LEFT_KNEE_x'], d['LEFT_KNEE_y']], [d['LEFT_ANKLE_x'], d['LEFT_ANKLE_y']])
        d['right_knee_angle'] = compute_angle([d['RIGHT_HIP_x'], d['RIGHT_HIP_y']], [d['RIGHT_KNEE_x'], d['RIGHT_KNEE_y']], [d['RIGHT_ANKLE_x'], d['RIGHT_ANKLE_y']])
        d['left_hip_angle'] = compute_angle([d['LEFT_SHOULDER_x'], d['LEFT_SHOULDER_y']], [d['LEFT_HIP_x'], d['LEFT_HIP_y']], [d['LEFT_KNEE_x'], d['LEFT_KNEE_y']])
        d['right_hip_angle'] = compute_angle([d['RIGHT_SHOULDER_x'], d['RIGHT_SHOULDER_y']], [d['RIGHT_HIP_x'], d['RIGHT_HIP_y']], [d['RIGHT_KNEE_x'], d['RIGHT_KNEE_y']])
    return d



In [ ]:
def initialize_video_processing(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    return cap, fps, pose

print('initialize_video_processing ready')


In [ ]:
def process_video_for_gait(video_path, cycles_per_video=5):
    cap, fps, pose = initialize_video_processing(video_path)
    frame_number = 0
    gait_count = 0
    recording_cycle = False
    current_cycle = []
    prev_left_heel_y = None
    prev_left_heel_pos = None
    all_rows = []

    while cap.isOpened() and gait_count < cycles_per_video:
        ret, frame = cap.read()
        if not ret:
            break
        frame_number += 1
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)
        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            row = extract_frame_landmarks(lm)

            left_heel_y = row.get('LEFT_HEEL_y')
            left_heel_pos = [row.get('LEFT_HEEL_x'), row.get('LEFT_HEEL_y')]

            if detect_heel_strike(prev_left_heel_y, left_heel_y):
                if not recording_cycle:
                    recording_cycle = True
                    current_cycle = []
                    cycle_start_frame = frame_number
                    prev_left_heel_pos = left_heel_pos
                else:
                    recording_cycle = False
                    gait_count += 1
                    cycle_end_frame = frame_number
                    stride_length = compute_stride_length(prev_left_heel_pos, left_heel_pos)
                    step_time_sec = (cycle_end_frame - cycle_start_frame) / fps
                    
                    for r in current_cycle:
                        r['stride_length'] = stride_length
                        r['step_time_sec'] = step_time_sec
                    all_rows.extend(current_cycle)

            prev_left_heel_y = left_heel_y

            if recording_cycle:
                current_cycle.append(row)

    cap.release()
    pose.close()
    return all_rows

In [ ]:
def extract_gait_features_to_csv(video_paths, output_csv='gait_features.csv', cycles_per_video=5, allowed_exts=None):
    allowed_exts = allowed_exts or ['.mp4', '.avi', '.mov', '.mkv']

    if isinstance(video_paths, (str, Path)):
        video_paths = [video_paths]

    all_rows = []
    video_items = []

    for p in video_paths:
        p = Path(p)
        if not p.exists():
            print(f'Warning: path does not exist, skipping: {p}')
            continue
        if p.is_dir():
            for child in sorted(p.iterdir()):
                if child.suffix.lower() in allowed_exts and child.is_file():
                    video_items.append(child)
        elif p.is_file():
            video_items.append(p)

    if not video_items:
        print('No video files found in provided paths. Writing empty CSV.')
        out_dir = Path('Datasets')
        out_dir.mkdir(parents=True, exist_ok=True)
        df = pd.DataFrame(all_rows)
        out_path = out_dir / output_csv
        df.to_csv(out_path, index=False)
        print(f'Saved {len(df)} rows to {out_path}')
        return df

    # Process each video and tag with gait_pattern (folder name)
    for video_path in video_items:
        folder_name = video_path.parent.name or ''
        try:
            rows = process_video_for_gait(str(video_path), cycles_per_video=cycles_per_video)
        except Exception as e:
            print(f'Error processing {video_path}: {e}')
            continue
        for r in rows:
            r['gait_pattern'] = folder_name
        all_rows.extend(rows)

    # Build DataFrame and save to Datasets/
    out_dir = Path('Datasets')
    out_dir.mkdir(parents=True, exist_ok=True)
    df = pd.DataFrame(all_rows)
    out_path = out_dir / output_csv
    df.to_csv(out_path, index=False)
    print(f'Saved {len(df)} rows to {out_path}')
    return df

In [ ]:

df_test = extract_gait_features_to_csv([], output_csv='test_empty_restored.csv')
print('Rows:', len(df_test))
print('Datasets files:', [p.name for p in Path('Datasets').glob('*')])


In [5]:
#This is to test the function for a single video
test_video = r"C:\Users\user\Desktop\Sem 1 Year 2\Artifical Intelligence\GiatLabDatset\Normal\001_NM_01.MOV"
saved_csv = process_video(test_video, "Normal")
print("Saved CSV:", saved_csv)

c:\Users\user\Desktop\GaitLab\GaitEnv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Saved CSV: keypoints_csv\001_NM_01_Normal.csv


In [6]:
# Cell 5: Process all videos in the dataset

for root, dirs, files in os.walk(VIDEO_ROOT):
    for file in tqdm(files):
        if file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            video_path = os.path.join(root, file)
            
            # Use the last folder name as label
            label = os.path.basename(os.path.dirname(video_path))
            
            saved_csv = process_video(video_path, label)
            if saved_csv:
                print("Saved:", saved_csv)


0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/26 [00:00<?, ?it/s]c:\Users\user\Desktop\GaitLab\GaitEnv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
  4%|▍         | 1/26 [00:09<04:06,  9.85s/it]

Saved: keypoints_csv\B01_Assistive.csv


c:\Users\user\Desktop\GaitLab\GaitEnv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
  8%|▊         | 2/26 [00:16<03:05,  7.73s/it]

Saved: keypoints_csv\B010_Assistive.csv


c:\Users\user\Desktop\GaitLab\GaitEnv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
 12%|█▏        | 3/26 [00:20<02:21,  6.13s/it]

...

Saved: keypoints_csv\001_PD_01_SV_PD_Severe.csv


c:\Users\user\Desktop\GaitLab\GaitEnv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
 40%|████      | 2/5 [08:02<10:34, 211.43s/it]

Saved: keypoints_csv\001_PD_02_SV_PD_Severe.csv


c:\Users\user\Desktop\GaitLab\GaitEnv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
 40%|████      | 2/5 [45:54<1:08:52, 1377.37s/it]


KeyboardInterrupt: 

In [10]:
import math

# --- Utility functions ---
def calculate_angle(a, b, c):
    """Compute angle (in degrees) between three 3D points a, b, c."""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ab, cb = a - b, c - b
    cosine = np.dot(ab, cb) / (np.linalg.norm(ab) * np.linalg.norm(cb) + 1e-6)
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

def calculate_distance(a, b):
    """Euclidean distance between two 3D points."""
    return np.linalg.norm(np.array(a) - np.array(b))


def extract_segment_features(csv_path, segments=5):
    df = pd.read_csv(csv_path)
    if df.empty:
        return []

    # Handle missing values safely
    df = df.interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill')
    if len(df.dropna()) < 10:
        print(f"Skipping {csv_path} (too few valid frames)")
        return []

    segment_size = len(df) // segments
    video_name = os.path.basename(csv_path)
    label = df["label"].iloc[0] if "label" in df.columns else "Unknown"

    all_segment_features = []

    for s in range(segments):
        seg = df.iloc[s * segment_size : (s + 1) * segment_size]
        if len(seg) < 3:
            continue

        numeric_cols = [c for c in df.columns if c.startswith(("x_", "y_", "z_"))]
        means = seg[numeric_cols].mean()
        stds = seg[numeric_cols].std()

        frame = seg.iloc[len(seg)//2]
        landmarks = np.array([[frame[f"x_{i}"], frame[f"y_{i}"], frame[f"z_{i}"]] for i in range(33)])

        # --- (1) Joint angles ---
        left_knee_angle = calculate_angle(landmarks[24], landmarks[26], landmarks[28])
        right_knee_angle = calculate_angle(landmarks[23], landmarks[25], landmarks[27])
        left_hip_angle = calculate_angle(landmarks[12], landmarks[24], landmarks[26])
        right_hip_angle = calculate_angle(landmarks[11], landmarks[23], landmarks[25])
        left_elbow_angle = calculate_angle(landmarks[12], landmarks[14], landmarks[16])
        right_elbow_angle = calculate_angle(landmarks[11], landmarks[13], landmarks[15])
        shoulder_tilt = calculate_angle(landmarks[11], landmarks[12], landmarks[24])

        # --- (2) Distances ---
        shoulder_width = calculate_distance(landmarks[11], landmarks[12])
        hip_width = calculate_distance(landmarks[23], landmarks[24])
        step_length = calculate_distance(landmarks[27], landmarks[28])
        torso_length = calculate_distance(landmarks[11], landmarks[23])
        leg_length_left = calculate_distance(landmarks[24], landmarks[28])
        leg_length_right = calculate_distance(landmarks[23], landmarks[27])

        # --- (3) Asymmetry features ---
        knee_angle_diff = abs(left_knee_angle - right_knee_angle)
        hip_angle_diff = abs(left_hip_angle - right_hip_angle)
        leg_length_ratio = leg_length_left / (leg_length_right + 1e-6)
        step_symmetry = abs(leg_length_left - leg_length_right)

        # --- (4) Posture ---
        com_x = np.mean([landmarks[11][0], landmarks[12][0], landmarks[23][0], landmarks[24][0]])
        com_y = np.mean([landmarks[11][1], landmarks[12][1], landmarks[23][1], landmarks[24][1]])
        posture_deviation = abs(landmarks[0][1] - com_y)

        # --- (5) Stability ratios ---
        shoulder_hip_ratio = shoulder_width / (hip_width + 1e-6)
        limb_ratio = (leg_length_left + leg_length_right) / (torso_length + 1e-6)

        biomech_dict = {
            "left_knee_angle": left_knee_angle,
            "right_knee_angle": right_knee_angle,
            "left_hip_angle": left_hip_angle,
            "right_hip_angle": right_hip_angle,
            "left_elbow_angle": left_elbow_angle,
            "right_elbow_angle": right_elbow_angle,
            "shoulder_tilt": shoulder_tilt,
            "shoulder_width": shoulder_width,
            "hip_width": hip_width,
            "step_length": step_length,
            "torso_length": torso_length,
            "leg_length_left": leg_length_left,
            "leg_length_right": leg_length_right,
            "knee_angle_diff": knee_angle_diff,
            "hip_angle_diff": hip_angle_diff,
            "leg_length_ratio": leg_length_ratio,
            "step_symmetry": step_symmetry,
            "posture_deviation": posture_deviation,
            "shoulder_hip_ratio": shoulder_hip_ratio,
            "limb_ratio": limb_ratio,
            "com_x": com_x
        }

        # Merge statistical features (mean/std) + biomechanical
        segment_features = {
            "video": video_name,
            "label": label,
            "segment": s
        }
        # add mean/std prefixed columns
        for col in means.index:
            segment_features[f"{col}_mean"] = means[col]
        for col in stds.index:
            segment_features[f"{col}_std"] = stds[col]

        # add biomechanical named features
        segment_features.update(biomech_dict)

        all_segment_features.append(segment_features)

    return all_segment_features


# --- Run over all extracted CSVs ---
final_features = []
for file in tqdm(os.listdir(OUTPUT_FOLDER)):
    if file.endswith(".csv"):
        csv_path = os.path.join(OUTPUT_FOLDER, file)
        feats = extract_segment_features(csv_path, segments=5)
        final_features.extend(feats)

final_df = pd.DataFrame(final_features)
final_csv_path = os.path.join(OUTPUT_FOLDER, "Final_Gait_Features_Named.csv")
final_df.to_csv(final_csv_path, index=False)

print(f"✅ Extracted {len(final_df)} samples with {final_df.shape[1]} features total.")
print(f"📁 Saved enriched dataset to: {final_csv_path}")



  0%|          | 0/252 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_10720\2496765150.py:22: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill')
C:\Users\user\AppData\Local\Temp\ipykernel_10720\2496765150.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill')
  0%|          | 1/252 [00:00<01:05,  3.85it/s]C:\Users\user\AppData\Local\Temp\ipykernel_10720\2496765150.py:22: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate(limit_direction='both').fillna(

✅ Extracted 1260 samples with 222 features total.
📁 Saved enriched dataset to: keypoints_csv\Final_Gait_Features_Named.csv
